<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/%EC%9D%BC%EB%B0%98%EB%8C%80%ED%99%94_%EC%B6%94%EA%B0%80_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 진행 순서

1. 데이터 불러오기
  - 일반 대화 데이터
  - 공격 대화 데이터
2. 키워드 추출
  - 공격 대화 데이터 키워드 추출
  - 일반 대화 데이터 키워드 추출
    - 화자 구분 x
    - 상담사 제외 고객만 키워드 추출
3. 키워드 비율에 따라 일반대화 추가

### 1. 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# 일반 대화 데이터
normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/일반대화/sns_data.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/Data/original_train_data.csv')

In [3]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화
...,...,...,...,...
3865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
3866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
3867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
3868,갈취 대화,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


In [4]:
# attack_conv.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv', index=False)

In [5]:
normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
26993,1:나 진짜 너무 피곤하다\n2:너 또 과제 하느라 밤 샜지\n1:우리 과는 시험 ...,나 진짜 너무 피곤하다\n너 또 과제 하느라 밤 샜지\n우리 과는 시험 대신 과제를...,일반 대화,일반 대화,SNS
26994,1:다들 요즘에 뭐 공부하고 있는 것 있어\n2:나는 특별히 공부하는 것은 없어\n...,다들 요즘에 뭐 공부하고 있는 것 있어\n나는 특별히 공부하는 것은 없어\n졸업시험...,일반 대화,일반 대화,SNS
26995,1:요즘 헬스장 다닌다며 잘 다니고 있어\n2:응 하루에 1 2시간 하고 있어\n1...,요즘 헬스장 다닌다며 잘 다니고 있어\n응 하루에 1 2시간 하고 있어\n유산소 위...,일반 대화,일반 대화,SNS
26996,1:요즘 날이 더운데 대중교통에 사람들도 많아서 너무 힘든 거 같아요\n2:네 너무...,요즘 날이 더운데 대중교통에 사람들도 많아서 너무 힘든 거 같아요\n네 너무 힘들어...,일반 대화,일반 대화,SNS


In [6]:
import numpy as np
idx = []
batch_idx = []

for i in range(len(normal_conv)):
  batch_idx.append(i)
  if len(batch_idx)==10:
    idx.append(batch_idx)
    batch_idx = []

In [7]:
len(idx)

2699

In [8]:
batch_sen = []
for i in idx:
  a = normal_conv.iloc[i].reset_index(drop=True).non_label_sentence
  sen = []
  for j in a:
    t = j.split('\n')
    sen += t

  batch_sen.append(sen)

In [9]:
batch_sen[0]

['야 니 남친은 잘 지내고 있니',
 '이야 한숨만 나온다',
 '엊그제 그래도 400일이었어',
 '와 너네가 벌써 400일이야',
 '그러게 정말 시간이 너무 빨라 키키',
 '어떻게 만나게 됐다고 했지',
 '유럽 여행에서 동행으로 만났어 키키',
 '걔는 몇 살 때 간 건데 그 나이에 유럽 여행을 갔대',
 '20살에 졸업하고 바로 왔나 봐 대단하더라',
 '와 그래도 여행 경비가 장난 아닐 텐데 그 돈은 어디서 모았대',
 '부모님이 졸업 선물로 해주신 것 같아',
 '키키 부럽다',
 '와 브루주아네 개부럽다',
 '나도 못 가 봤는데',
 '걔랑 2주 정도 같이 있었던 것 같아',
 '야 그렇게 여행 경로가 겹쳤었어',
 '응 우리가 상의하고 계획했어',
 '헐 그때부터 서로 마음에 들었나 보네 유후',
 '어머나  들켰다 내 큰 그림이지',
 '나 궁금한 게 하나 있어',
 '응 그게 뭔데 자기야',
 '작은 아빠는 풍산개를 어디서 그렇게 데리고 오는 거야',
 '아 그거 아는 지인이 애견 훈련소 했어',
 '헐 대박 지금은 안 하고 계셔',
 '음    글쎄 잘 모르겠네',
 '그 풍산개 혈통 있는 개라고 막 그랬잖아',
 '어 맞아  동물 농장에 나오신 훈련사 분이 선물 해줬어 키키',
 '헐 대박 혹시 내가 아는 그분이니',
 '응 맞아 그런 것 같아',
 '헐 대박 사건   근데 우리 진이 목줄 풀고 가출했어',
 '역시 우리 진이 자유를 동경했구나 키키',
 '알지 우리 집 주변에 개 농장 있는 거',
 '헐 어떡해',
 '끌려간 거 아니야',
 '주변에 다 봤어',
 '그때 한창 뉴스에서 개 도둑 많다고 조심하라고 했는데',
 '아이고 우리 진이야 아이고 우리 진이야',
 '너 최근에 했던 운동이 뭐야',
 '나 등산 갔었지',
 '관악산 올라가 봤나',
 '관악산이 어디 있는 산이야',
 '관악구에 있는 산이지',
 '언니네 집 근처야 키키',
 '아아 좋은 정보 감사요',
 '그렇다고 가진 않을 거야',
 '너무 단호하네',
 '

In [10]:
len(batch_sen)

2699

### 2. 키워드 추출

1. 공격 대화 키워드 추출


In [11]:
!pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
def keyword(class_name, data, n=20):
    genre = data[data['class']==class_name].reset_index(drop=True).label_sentence

    attack, damaged = [], []

    for e in genre:
        e = e.split('\n')
        for e_ in e:
            if e_[0] == '1':
                attack.append(e_[2:])
            else:
                damaged.append(e_[2:])

    from krwordrank.sentence import summarize_with_sentences

    texts = attack
    keywords_att, sents = summarize_with_sentences(texts, num_keywords=50, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    texts = damaged
    keywords_dam, sents = summarize_with_sentences(texts, num_keywords=50, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    print('{}의 가해자 :'.format(class_name),keywords_att.keys())
    print()
    print('{}의 피해자 :'.format(class_name),keywords_dam.keys())
    
    keywords_att = list(keywords_att.keys())
    keywords_dam = list(keywords_dam.keys())
    
    target_attack, target_damaged = [], []

    for i in range(50):
        if (keywords_att[i] not in keywords_dam) : target_attack.append(keywords_att[i])
        if len(target_attack) == n: break 

    for i in range(50):
        if (keywords_dam[i] not in keywords_att) : target_damaged.append(keywords_dam[i])
        if len(target_damaged) == n: break
    
    print()
    print('{}의 가해자(중복제거) :'.format(class_name),target_attack)
    print()
    print('{}의 피해자(중복제거) :'.format(class_name),target_damaged)

    return [target_attack, target_damaged]

In [13]:
extort_att, extort_vic = keyword('갈취 대화', attack_conv)

갈취 대화의 가해자 : dict_keys(['내가', '아니', '그럼', '그래', '빌려', '지금', '그냥', '내놔', '이거', '네가', '원만', '빨리', '진짜', '우리', '친구', '돈이', '거야', '오늘', '없어', '있는', '그러', '가져', '나도', '나오면', '말고', '전에', '제가', '있어', '여기', '사람', '보내', '좋은', '10', '너네', '알아', '너무', '이번', '거기', '새끼', '엄마', '어떻게', '그거', '얼마', '당장', '이렇게', '그렇게', '내일', '뒤져서', '말이', '나한테'])

갈취 대화의 피해자 : dict_keys(['아니', '제발', '진짜', '없어', '안돼', '무슨', '이거', '내가', '돈이', '정말', '미안', '그래', '제가', '그러', '거야', '알겠', '그건', '여기', '어떻게', '지금', '죄송', '엄마', '빌려', '돌려', '그런', '이번', '나도', '그럼', '없는', '이게', '이러', '저희', '그거', '이건', '있어', '그게', '저번', '거예요', '우리', '말이', '너무', '마세요', '가져', '오늘', '주세요', '그렇', '빨리', '얼마', '드릴', '어디'])

갈취 대화의 가해자(중복제거) : ['그냥', '내놔', '네가', '원만', '친구', '있는', '나오면', '말고', '전에', '사람', '보내', '좋은', '10', '너네', '알아', '거기', '새끼', '당장', '이렇게', '그렇게']

갈취 대화의 피해자(중복제거) : ['제발', '안돼', '무슨', '정말', '미안', '알겠', '그건', '죄송', '돌려', '그런', '없는', '이게', '이러', '저희', '이건', '그게', '저번', '거예요', '마세요', '주세요']


In [14]:
work_att, work_vic = keyword('직장 내 괴롭힘 대화', attack_conv)

직장 내 괴롭힘 대화의 가해자 : dict_keys(['내가', '아니', '거야', '그래', '지금', '오늘', '자네', '그럼', '우리', '그렇게', '하는', '사람', '그냥', '이렇게', '무슨', '네가', '김대리', '이번', '이거', '하고', '일을', '진짜', '그러', '그런', '말이', '없어', '있어', '어떻게', '대리', '말고', '요즘', '너무', '이런', '하나', '일이', '내일', '다른', '생각', '죄송', '알아', '제대로', '보고', '휴가', '아주', '있는', '빨리', '누가', '못하', '하면', '같은'])

직장 내 괴롭힘 대화의 피해자 : dict_keys(['제가', '죄송', '아니', '부장님', '무슨', '아닙니다', '너무', '과장님', '정말', '다시', '말씀', '지금', '일이', '그건', '저도', '하겠습니다', '그래도', '그런', '오늘', '팀장님', '알겠', '그게', '저는', '이건', '대리님', '업무', '있어서', '열심히', '사장님', '같습니', '그러', '아닌', '빨리', '있습니다', '다른', '그럼', '그렇', '내일', '진짜', '휴가', '없습니다', '같아', '이번', '잘못', '앞으로', '하고', '어떻게', '있는', '바로', '아직'])

직장 내 괴롭힘 대화의 가해자(중복제거) : ['내가', '거야', '그래', '자네', '우리', '그렇게', '하는', '사람', '그냥', '이렇게', '네가', '김대리', '이거', '일을', '말이', '없어', '있어', '대리', '말고', '요즘']

직장 내 괴롭힘 대화의 피해자(중복제거) : ['제가', '부장님', '아닙니다', '과장님', '정말', '다시', '말씀', '그건', '저도', '하겠습니다', '그래도', '팀장님', '알겠', '그게', '저는', '이건', '대리님', '업무', '있어서', '열심히']


In [15]:
threat_att, threat_vic = keyword('협박 대화', attack_conv)

협박 대화의 가해자 : dict_keys(['내가', '거야', '네가', '지금', '아니', '진짜', '그래', '그냥', '그럼', '죽여', '우리', '사람', '있어', '알아', '하나', '그러', '죽어', '어떻게', '전에', '당장', '죽고', '새끼', '어디', '정신', '빨리', '당신', '없어', '하면', '너도', '그렇게', '너네', '무슨', '나도', '이렇게', '여기', '싶어', '가족', '있는', '오늘', '이게', '나한테', '생각', '이제', '아주', '헤어', '같이', '말이', '들어', '하는', '않으면'])

협박 대화의 피해자 : dict_keys(['제발', '아니', '무슨', '정말', '내가', '제가', '우리', '거야', '죄송', '살려', '그러', '진짜', '지금', '그래', '잘못', '어떻게', '네가', '미안', '너무', '그런', '이러', '사람', '주세요', '그만', '그렇게', '그냥', '경찰', '알겠', '말아', '그건', '마세요', '그게', '이제', '거예요', '시간', '신고', '없어', '협박', '소리', '이거', '조금만', '나는', '당신', '아닙니다', '선생님', '하지', '하는', '여기', '그럼', '저는'])

협박 대화의 가해자(중복제거) : ['죽여', '있어', '알아', '하나', '죽어', '전에', '당장', '죽고', '새끼', '어디', '정신', '빨리', '하면', '너도', '너네', '나도', '이렇게', '싶어', '가족', '있는']

협박 대화의 피해자(중복제거) : ['제발', '정말', '제가', '죄송', '살려', '잘못', '미안', '너무', '그런', '이러', '주세요', '그만', '경찰', '알겠', '말아', '그건', '마세요', '그게', '거예요', '시간']


In [16]:
other_att, other_vic = keyword('기타 괴롭힘 대화', attack_conv)

기타 괴롭힘 대화의 가해자 : dict_keys(['아니', '내가', '네가', '진짜', '그냥', '그래', '우리', '거야', '그럼', '여기', '지금', '너무', '무슨', '그렇게', '사람', '그러', '새끼', '이렇게', '어디', '빨리', '이거', '어떻게', '그런', '너네', '하는', '얼굴', '친구', '장애', '냄새', '말고', '거지', '같은', '뭐가', '엄마', '돼지', '다니', '있어', '것도', '너는', '없어', '못생', '하지', '알아', '기분', '하고', '이게', '정말', '이런', '나는', '남자'])

기타 괴롭힘 대화의 피해자 : dict_keys(['아니', '무슨', '내가', '손님', '그만', '제발', '거야', '제가', '죄송', '그래', '그런', '너무', '진짜', '그러', '미안', '지금', '그렇', '말이', '우리', '하지', '그냥', '나도', '그게', '정말', '네가', '있어', '저희', '하는', '그건', '여기', '엄마', '사람', '어떻게', '이거', '나한', '기분', '잘못', '그럼', '이러', '선생님', '다른', '말씀', '다시', '자꾸', '환불', '혹시', '하고', '오늘', '너네', '얘들아'])

기타 괴롭힘 대화의 가해자(중복제거) : ['그렇게', '새끼', '이렇게', '어디', '빨리', '얼굴', '친구', '장애', '냄새', '말고', '거지', '같은', '뭐가', '돼지', '다니', '것도', '너는', '없어', '못생', '알아']

기타 괴롭힘 대화의 피해자(중복제거) : ['손님', '그만', '제발', '제가', '죄송', '미안', '그렇', '말이', '나도', '그게', '저희', '그건', '나한', '잘못', '이러', '선생님', '다른', '말씀', '다시', '자꾸']


위 키워드를 바탕으로 중복을 제거한 가해자들의 키워드를 따로 저장해줌.

피해자는 대부분 `네? 죄송합니다. 미안 ` 등의 키워드가 나와서 비슷한 모습을 볼 수 있음.

공격적인 내용의 대부분은 가해자가 하는 말에의해 결정된다고 볼 수 있음.

In [17]:
att_keyword = extort_att + work_att + threat_att + other_att

In [18]:
# 중복 제거
att_keyword = list(set(att_keyword))
len(att_keyword)

57

In [19]:
batch_sen[1]

['나 당근 마켓에 화장품 올렸던 거 알아',
 '당근 마켓이 뭐야',
 '그런 건 처음 듣는데  마트야',
 '너 어린 애가 당근 마켓도 모르니',
 '그러게 나 문찐인가 봐 키키',
 '뭐 하는 건데',
 '동네 사람들이랑 거래하는 거야',
 '아  택배 거래 아니고 직거래',
 '어 근데 택배 거래도 상대방이랑 얘기해서 가능해',
 '오 괜찮다 키키',
 '그렇지만 무섭지 않아',
 '돈 들고 튀면 어떡해',
 '그래서 되도록 현금 가져오라고 하지',
 '오    계좌 이체 하는 건 아니고',
 '응 그것도 해도 되고 나는 되도록 현금 추구함',
 '그렇구나 화장품 뭐 팔았어',
 '나 몰래 내 꺼 판 거 아니지',
 '그럴 리가  올영에서 샀는데 양이 많아서 하나 올렸지',
 '그런 거 있으면 나나 주지 왜 팔아',
 '아니  너랑 사는 곳이 멀잖아',
 '그건 인정 당근 마켓 유용하네 키키',
 '나 몰랐는데 박보검 군대 갔니',
 '어휴 간 지가 언젠데  우리 오빠 열일 하잖아',
 '맞아 해군으로 갔다고 하더라',
 '그래 사진 봤어',
 '정복 잘 어울려 보검씨',
 '야 사람들이 박보검 잘생겼다고 했잖아',
 '이제 알았잖아 그거 보고 키키',
 '나 무슨 군대 드라마 한편 본 줄 알았어 키키',
 '진짜 잘생겼더라',
 '나는 한 장밖에 못 봤는데 여러 장 있니',
 '공유 좀',
 '네이버 기사에 많이 올라 오더라',
 '확실히 박보검 최고',
 '보통 연예인들 군대 사진 보면 일반 사람 같던데 박보검은 진짜 연예인이더라 키키',
 '어 맞아 군대 사진으로 옥택연도 유명한데 봤어',
 '옥택연은 못 봤는데 그 사람도 해군이야',
 '아니 육군으로 간 것 같더라',
 '피지컬이 더 좋아졌어',
 '육군이면 이끼룩 입었겠네 전역했어',
 '전역한 것 같던데  요즘 티비에 자주 봐',
 '아 진짜  나는 요즘에 티비를 안 봐서 몰랐네',
 '티비도 좀 보고 살아 언니',
 '너의 최애 영화는 뭐야',
 '나 작년에 나오 테넷이 내 최애 영화야',

In [20]:
def select_normal_conversation(sen, att_keyword, threshold=0.2):

  
  from krwordrank.sentence import summarize_with_sentences

  texts = sen
  keywords_client, sents = summarize_with_sentences(texts, num_keywords=30, 
                                           num_keysents=5, max_length=10, diversity=0.5)

  keywords_client = list(keywords_client.keys())
  keywords_att = att_keyword

  length = len(keywords_client)

  clean_keywords = []
  for word in keywords_client:
    if not word in keywords_att:
      clean_keywords.append(word)
  
  clean_ratio = len(clean_keywords) / length
  if clean_ratio > threshold:
    return True
  else:
    return False

In [21]:
select_batch_idx = []
attack_batch_idx = []

for i in range(len(batch_sen)):
  try:
    if select_normal_conversation(batch_sen[i], att_keyword):
      select_batch_idx.append(i)
    else:
      attack_batch_idx.append(i)
  except:
    continue

In [22]:
len(attack_batch_idx)

1

In [23]:
normal_idx = []
for i in select_batch_idx:
  normal_idx += idx[i]

attack_idx = []
for i in attack_batch_idx:
  attack_idx += idx[i]

In [24]:
len(normal_idx), len(attack_idx)

(26970, 10)

In [25]:
filtered_attack_conv = normal_conv.iloc[attack_idx]
filtered_attack_conv

,label_sentence,non_label_sentence,class,binary_class,type
24170,1:우리 아이 오늘 부터 미술학원 다녀요\n2:축하해요 좋아하던가요\n1:너무 좋...,우리 아이 오늘 부터 미술학원 다녀요\n축하해요 좋아하던가요\n너무 좋아 하더라구...,일반 대화,일반 대화,SNS
24171,1:나 우리 엄마 자동차 사드렸어\n2:우리 어제 가족 사진을 찍었어 오랜만이라 좋...,나 우리 엄마 자동차 사드렸어\n우리 어제 가족 사진을 찍었어 오랜만이라 좋더라고\...,일반 대화,일반 대화,SNS
24172,1:국제결혼 하는 거 어때보여\n2:본인의 선택인 거 같아 난 찬성\n1:근처에 ...,국제결혼 하는 거 어때보여\n본인의 선택인 거 같아 난 찬성\n근처에 국제 결혼 ...,일반 대화,일반 대화,SNS
24173,1:너네들 강아지 키워\n2:응 지난 달부터 키우기 시작했어\n1:아하 나는 키...,너네들 강아지 키워\n응 지난 달부터 키우기 시작했어\n아하 나는 키운지 얼마 ...,일반 대화,일반 대화,SNS
24174,1:형부가 낚시줄에 바늘을 끼고 있어서 뭐하는지 물어봤어요\n2:고기 밥 준비하시는...,형부가 낚시줄에 바늘을 끼고 있어서 뭐하는지 물어봤어요\n고기 밥 준비하시는 거였나...,일반 대화,일반 대화,SNS
24175,1:배틀그라운드 알아\n2:나는 생소하네\n1:그런거야 요즘 인기인가봐\n2:요즘...,배틀그라운드 알아\n나는 생소하네\n그런거야 요즘 인기인가봐\n요즘 많이 한데\n...,일반 대화,일반 대화,SNS
24176,1:요즘 이상기후 문제야\n2:맞아 너무 추워지고 더워져\n1:그러게 환경 보전...,요즘 이상기후 문제야\n맞아 너무 추워지고 더워져\n그러게 환경 보전 신경써야 ...,일반 대화,일반 대화,SNS
24177,1:나 방금 뉴스보다가 어처구니 없는거 봤다\n2:뭔데 뭔데 뭔데\n1:제주도 빵집...,나 방금 뉴스보다가 어처구니 없는거 봤다\n뭔데 뭔데 뭔데\n제주도 빵집에서 어떤 ...,일반 대화,일반 대화,SNS
24178,1:요즘 피부 탄력이 많이 떨어진가 같아서 콜라겐을 먹어볼까 하는데 어때\n2:난 ...,요즘 피부 탄력이 많이 떨어진가 같아서 콜라겐을 먹어볼까 하는데 어때\n난 콜라겐 ...,일반 대화,일반 대화,SNS
24179,1:재밌는 책 추천해 줘\n2: 봉제인형 살인사건 알아\n1:그게 무슨 책이야\n...,재밌는 책 추천해 줘\n봉제인형 살인사건 알아\n그게 무슨 책이야\n추리소설이야\...,일반 대화,일반 대화,SNS


In [33]:
print(filtered_attack_conv['label_sentence'][24170])

1:우리 아이 오늘 부터 미술학원 다녀요
2:축하해요  좋아하던가요
1:너무 좋아 하더라구요 예전부터 보내 달라 했었는데 이제 겨우 보냈거든요
2:우리 아이도 가고 싶어하는데 못 보내고 있어요
1:그나마 저도 겨우 간다고 하는곳이 미술이네요
2:가고 싶다고 하는 곳이 있어서 좋네요
1:저희도 무리해서 보내는거라서 더 간다고 할까봐 무서울 때가 있네요
2:한다고 하면 다 시켜줄텐데 의욕이 없네요


In [34]:
print(filtered_attack_conv['label_sentence'][24171])

1:나 우리 엄마 자동차 사드렸어
2:우리 어제 가족 사진을 찍었어 오랜만이라 좋더라고
1:지난번에 찍으러 간다고 하지 않았어  그 날 못 갔구나
2:어 그래서 어제 찍었어 좋더라고
1:자동차 사드리니 너무 좋아하시더라고
1:그래서 엄마랑 하루종일 운전 연습하고 들어왔어
2:좋겠다 엄마랑 운전도 같이 연습하고 보기 좋다
1:너무 좋아하시더라고  화낼 뻔 했지만 참았어
2:엄마랑 데이트 엄청 부럽다
2:우리 모녀는 운전 할수 있는 사람이 없어서 아쉽네


In [35]:
print(filtered_attack_conv['label_sentence'][24172])

1:국제결혼 하는 거 어때보여
2:본인의 선택인 거 같아  난 찬성
1:근처에 국제 결혼 한 커플이 있어
2:물론 언어적인 문제도 있겠지만 더러 잘 살더라
1:나는 파키스탄인과 인도인과 결혼한 친구들 있는데 다들 잘 살더라
2:요즘 유튜브 보다보면 그런 커플들 이야기가 많아
1:베트남에서 한베가정은 행복해보였어
2:일반적으로는 불행해보였어
2:뭔가 겉도는 느낌이라고 할까


In [36]:
print(filtered_attack_conv['label_sentence'][24173])

1:너네들 강아지 키워
2:응  지난 달부터 키우기 시작했어
1:아하  나는 키운지 얼마 안 됐어
2:정말 귀엽겠다  배변 훈련은 했어
1:너희들은 어떻게 훈련했어
2:나는 배변패드에서 볼일을 보면 칭찬을 했어
1:정말  쉽지 않겠다
2:응  아직도 배변패드에서 볼일을 안 보는 경우가 많아


In [37]:
print(filtered_attack_conv['label_sentence'][24174])

1:형부가 낚시줄에 바늘을 끼고 있어서 뭐하는지 물어봤어요
2:고기 밥 준비하시는 거였나요
1:11월달에 또 갈치 낚시를 간다고 하더라고요
2:춥지 않나요
1:몇개월에 한번씩 바다 낚시를 가는 형부가 이해가 안가요
2:이야기를 들어보니 저도 이해가 안 갑니다
1:가서 생고생만 하고 오는데도 좋아하더라고요
2:집나가면 생고생인데 다들 왜 그러시는지


In [39]:
print(filtered_attack_conv['label_sentence'][24175])

1:배틀그라운드 알아
2:나는 생소하네
1:그런거야  요즘 인기인가봐
2:요즘 많이 한데
1:난 게임을 잘 몰라서 궁금해
2:젊은 사람들이 하는 건가 보다
1:총 쏘는 건 싫은데
2:온라인으로 하는 건가


In [40]:
print(filtered_attack_conv['label_sentence'][24176])

1:요즘 이상기후 문제야
2:맞아 너무 추워지고 더워져
1:그러게   환경 보전 신경써야 되는데
2:그렇지 환경 오염이 문제지
1:이 가을에 단풍도 없는거 같아
2:헉 가을에 단풍이 없다니 슬프다
1:단풍이 안이쁘데
2:코로나로 축제도 다 없데


In [41]:
print(filtered_attack_conv['label_sentence'][24177])

1:나 방금 뉴스보다가 어처구니 없는거 봤다
2:뭔데 뭔데 뭔데
1:제주도 빵집에서 어떤 할머니가 문을 못열어서 보다못한 청년이 문을 대신 열어줬거든
2:착한사람이네
2:좋은 일 했구만
1:근데 그 할머니가 균형을 잃고 넘어지면서 뇌손상이 와서 사망하셨고 청년은 과실치사로 입건됐대
2:헐  도와주려다가 감옥가게 되는거야
1:모르겠어
1:사람이 죽고 유족들이 신고하면 입건이 안될수가 없다나봐
2:참 답답하네 뭔가


In [42]:
print(filtered_attack_conv['label_sentence'][24178])

1:요즘 피부 탄력이 많이 떨어진가 같아서 콜라겐을 먹어볼까 하는데 어때
2:난 콜라겐 먹고 있어
1:정말  언니가 먹는 콜라겐은 뭐야
2:요즘은 먹기편하게 젤리로도 나오더라고
1:응 난 알약보다 가루가 더 맞는거같아
1:젤리형태로 된 콜라겐도 괜찮겠다
2:응 후기보면 맛도 있어서 약같이 느껴지지않는다하네
1:맛도 중요하지만 효과가있어야하니 후기를 자세히 봐야겠어
2:응 꼼꼼히 찾아봐


In [43]:
print(filtered_attack_conv['label_sentence'][24179])

1:재밌는 책 추천해 줘
2: 봉제인형 살인사건  알아
1:그게 무슨 책이야
2:추리소설이야
1:나 다니엘 콜 작품 다 좋아해
2:그럼  봉제인형 살인사건  꼭 읽어
1:그건 언제 출시했어
2:올해 2월에 출시했어


In [28]:
df = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_norm_data.csv')
df

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,네 침구 추가를 해야 할 것 같아서요\n네 1세트가 필요하신 건가요\n네 맞습니다 ...,1:반갑습니다 입니다\n2:네 침구 추가를 해야 할 것 같아서요\n1:네 1세트가 ...,일반 대화,일반 대화
1,고객센터,네 예약하려면 어떻게 해야 하나요\n네 회원 가입 후 로그인 하셔서 예약하시면 되세...,1:반갑습니다 입니다\n2:네 예약하려면 어떻게 해야 하나요\n1:네 회원 가입 후...,일반 대화,일반 대화
2,고객센터,네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n아 일반적인 스탠다드 디럭스 객...,1:반갑습니다 입니다\n2:네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n1:...,일반 대화,일반 대화
3,고객센터,네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n아 코인 런더리룸이요\n네 어디에...,1:반갑습니다 입니다\n2:네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n1:아...,일반 대화,일반 대화
4,고객센터,네 혹시 부대 시설로 편의점이 있나요\n네 호텔 1층에 위치해 있습니다\n아 어떤 ...,1:반갑습니다 입니다\n2:네 혹시 부대 시설로 편의점이 있나요\n1:네 호텔 1층...,일반 대화,일반 대화
...,...,...,...,...,...
106225,카카오,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일반 대화,일반 대화
106226,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화
106227,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
106228,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화


In [29]:
norm_data = pd.concat([filtered_normal_conv, df], ignore_index=True)
norm_data

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
133195,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,일반 대화,일반 대화,카카오
133196,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,일반 대화,일반 대화,카카오
133197,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,일반 대화,일반 대화,카카오
133198,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,일반 대화,일반 대화,카카오


In [30]:
# norm_data.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_sns_data.csv', index=False)

### 3. 비율에 따라 일반대화 추가

먼저 카카오톡 대화와 고객센터 대화의 비율은 3:1로 정했습니다.

기존의 공격대화에서 볼 때 직장 내 괴롭힘 클래스에 해당하는 대화를 제외하고는 대부분 가벼운 말투(야, 너, 가져와, 해라 등등)입니다.

추가할 일반 데이터인 카카오톡 데이터와 고객센터 데이터를 비교해보면 카카오톡 데이터가 비교적 가벼운 말투로 이루어져있고, 고객센터 대화가 조금 더 딱딱한 말투로 이루어져 있기 때문에 이에 맞춰서 데이터의 비율을 정했습니다.

먼저 공격대화를 일반 대화와 비슷하게 한글 숫자 제외하고 전부 전처리 해줍니다.

In [31]:
import pandas as pd
import numpy as np
import re

# 일반 대화 데이터
normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_sns_data.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/Augmentation/train&back_en2ko_fr2ko.csv')
attack_conv2 = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/back_translation/train_bt_ch2ko.csv')


FileNotFoundError: ignored

In [ ]:
normal_conv

In [ ]:
attack_conv

In [ ]:
attack_conv2

In [ ]:
attack_conv = pd.concat([attack_conv, attack_conv2], ignore_index=True)
attack_conv

In [ ]:
def preprocess(sen):
  sen = sen.split('\n')
  
  clean_sen = []
  for i in sen:
    i = i.strip()
    i = re.sub(r'[^0-9가-힣:]', ' ', i)
    i = i.strip()
    clean_sen.append(i)

  return '\n'.join(clean_sen)

In [ ]:
normal_conv['label_sentence'] = normal_conv['label_sentence'].apply(lambda x : preprocess(x))
normal_conv

In [ ]:
normal_conv['non_label_sentence'] = normal_conv['non_label_sentence'].apply(lambda x : preprocess(x))
normal_conv

In [ ]:
attack_conv['non_label_sentence'] = attack_conv['non_label_sentence'].apply(lambda x : preprocess(x))
attack_conv

In [ ]:
attack_conv['non_label_sentence'].duplicated().sum()

In [ ]:
attack_conv.drop_duplicates(['non_label_sentence'], keep='first', inplace=True)
attack_conv['non_label_sentence'].duplicated().sum()

1. 일반 대화 1000개 추가 (카카오톡 750 : 고객센터 250)

In [ ]:
norm_2000 = normal_conv.sample(4000)
norm_2000

In [ ]:
train_rs_2000 = pd.concat([norm_2000.iloc[:, :-1], attack_conv], ignore_index=True)
train_rs_2000

In [ ]:
# train_rs_2000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_en2ko_fr2ko_ch2ko_4000.csv', index=False)

In [ ]:
train_rs2_2000 = pd.concat([norm_2000.iloc[:, :-1], attack_conv2], ignore_index=True)
train_rs2_2000

In [ ]:
# train_rs2_2000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_en2ko_2000.csv', index=False)

In [ ]:
norm_4000 = normal_conv.sample(4000)
norm_4000

In [ ]:
train_rs2_4000 = pd.concat([norm_4000.iloc[:, :-1], attack_conv3], ignore_index=True)
train_rs2_4000

In [ ]:
# train_rs2_4000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_en2ko_ch2ko_4000.csv', index=False)

In [ ]:
train_4000 = pd.concat([norm_4000.iloc[:,1:], attack_conv])
train_4000

In [ ]:
# train_4000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_bt_4000.csv', index=False)

3. 일반 대화 10000개 추가 (카카오 7500 : 고객센터 2500)

In [ ]:
kakao_7500 = kakao_data.sample(7500)
kakao_7500

In [ ]:
cs_2500 = cs_data.sample(2500)
cs_2500

In [ ]:
norm_10000 = pd.concat([kakao_7500, cs_2500], ignore_index=True)
norm_10000

In [ ]:
train_10000 = pd.concat([norm_10000.iloc[:, 1:], attack_conv], ignore_index=True)
train_10000

In [ ]:
# train_10000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_bt_10000.csv', index=False)